In [1]:
import os
import azureml.core
from azureml.core import Workspace, Dataset, Datastore, ComputeTarget, Experiment, ScriptRunConfig
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.37.0


In [2]:
workspace = Workspace.from_config()
print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep='\n')

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
Workspace name: basic-ws-2
Azure region: eastus
Subscription id: 790907ce-38df-431c-9fce-11b9ec68781d
Resource group: test


In [3]:
# create an ML experiment
exp = Experiment(workspace=workspace, name='keras-mnist-fashion')

# create a directory
script_folder = './keras-mnist-fashion'
os.makedirs(script_folder, exist_ok=True)

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())


Creating a new compute target...
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2021-12-14T00:11:49.623000+00:00', 'errors': None, 'creationTime': '2021-12-14T00:11:49.166964+00:00', 'modifiedTime': '2021-12-14T00:11:54.760273+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [5]:
data_urls = ['https://data4mldemo6150520719.blob.core.windows.net/demo/mnist-fashion']
fashion_ds = Dataset.File.from_files(data_urls)

# list the files referenced by fashion_ds
fashion_ds.to_path()


['/mnist-fashion/t10k-images-idx3-ubyte',
 '/mnist-fashion/t10k-labels-idx1-ubyte',
 '/mnist-fashion/train-images-idx3-ubyte',
 '/mnist-fashion/train-labels-idx1-ubyte']

In [6]:
from azureml.data import OutputFileDatasetConfig

datastore=workspace.get_default_datastore()
prepared_fashion_ds = OutputFileDatasetConfig(destination=(datastore, 'outputdataset/{run-id}')).register_on_complete(name='prepared_fashion_ds')


In [7]:
prep_step = PythonScriptStep(name='prepare step',
                             script_name="prepare.py",
                             # mount fashion_ds dataset to the compute_target
                             arguments=[fashion_ds.as_named_input('fashion_ds').as_mount(), prepared_fashion_ds],
                             source_directory=script_folder,
                             compute_target=compute_target,
                             allow_reuse=True)

In [8]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- pip:
  - azureml-core
  - azureml-dataset-runtime
  - keras==2.4.3
  - tensorflow==2.4.3
  - numpy
  - scikit-learn
  - pandas
  - matplotlib

Writing conda_dependencies.yml


In [9]:
from azureml.core import Environment

keras_env = Environment.from_conda_specification(name = 'keras-env', file_path = './conda_dependencies.yml')

In [10]:
train_src = ScriptRunConfig(source_directory=script_folder,
                            script='train.py',
                            compute_target=compute_target,
                            environment=keras_env)

In [11]:


train_step = PythonScriptStep(name='train step',
                              arguments=[prepared_fashion_ds.read_delimited_files().as_input(name='prepared_fashion_ds')],
                              source_directory=train_src.source_directory,
                              script_name=train_src.script,
                              runconfig=train_src.run_config)



In [12]:


# build pipeline & run experiment
pipeline = Pipeline(workspace, steps=[prep_step, train_step])
run = exp.submit(pipeline)



Created step prepare step [39b13515][877685a0-4903-438a-8a30-0261f380e544], (This step will run and generate new outputs)Created step train step [600ab4e1][8ff6582c-1c97-4741-b72b-8d8cde628cee], (This step will run and generate new outputs)

Submitted PipelineRun 39236fa9-d0ba-46c0-a840-02debd38aa04
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/39236fa9-d0ba-46c0-a840-02debd38aa04?wsid=/subscriptions/790907ce-38df-431c-9fce-11b9ec68781d/resourcegroups/test/workspaces/basic-ws-2&tid=5100e2c5-9fd5-492c-8e65-b9e2a0d8584f


In [13]:
run.wait_for_completion(show_output=True)

PipelineRunId: 39236fa9-d0ba-46c0-a840-02debd38aa04
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/39236fa9-d0ba-46c0-a840-02debd38aa04?wsid=/subscriptions/790907ce-38df-431c-9fce-11b9ec68781d/resourcegroups/test/workspaces/basic-ws-2&tid=5100e2c5-9fd5-492c-8e65-b9e2a0d8584f
PipelineRun Status: Running


StepRunId: 55844d80-1a7a-4532-bac2-8c68f1197a86
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/55844d80-1a7a-4532-bac2-8c68f1197a86?wsid=/subscriptions/790907ce-38df-431c-9fce-11b9ec68781d/resourcegroups/test/workspaces/basic-ws-2&tid=5100e2c5-9fd5-492c-8e65-b9e2a0d8584f
StepRun( prepare step ) Status: Running

Streaming azureml-logs/20_image_build_log.txt
2021/12/14 00:13:02 Downloading source code...
2021/12/14 00:13:03 Finished downloading source code
2021/12/14 00:13:03 Creating Docker network: acb_default_network, driver: 'bridge'
2021/12/14 00:13:04 Successfully set up Docker network: acb_default_network
2021/12/14 00:13:04 Setting up Docker co

'Finished'

In [14]:
run.find_step_run('train step')[0].get_metrics()

{'Loss': [0.8421418070793152,
  0.5300726294517517,
  0.4550732970237732,
  0.4121553599834442,
  0.3826228380203247,
  0.3648979365825653,
  0.34244516491889954,
  0.33340883255004883,
  0.319690465927124,
  0.31032752990722656],
 'Accuracy': [0.684268057346344,
  0.802508533000946,
  0.8324369192123413,
  0.8477693200111389,
  0.8613832592964172,
  0.8654897809028625,
  0.8735018968582153,
  0.8778315782546997,
  0.8825852870941162,
  0.8860669136047363],
 'Final test loss': 0.2672939896583557,
 'Final test accuracy': 0.9014225602149963,
 'Loss v.s. Accuracy': 'aml://artifactId/ExperimentRun/dcid.bbd04b2a-d9a0-44d8-af56-6efd5c34f50a/Loss v.s. Accuracy_1639441942.png'}

In [15]:
# get input datasets
prep_step = run.find_step_run('prepare step')[0]
inputs = prep_step.get_details()['inputDatasets']
input_dataset = inputs[0]['dataset']

# list the files referenced by input_dataset
input_dataset.to_path()

['/mnist-fashion/t10k-images-idx3-ubyte',
 '/mnist-fashion/t10k-labels-idx1-ubyte',
 '/mnist-fashion/train-images-idx3-ubyte',
 '/mnist-fashion/train-labels-idx1-ubyte']

In [16]:
fashion_ds = input_dataset.register(workspace = workspace,
                                    name = 'fashion_ds',
                                    description = 'image and label files from fashion mnist',
                                    create_new_version = True)
fashion_ds


{
  "source": [
    "https://data4mldemo6150520719.blob.core.windows.net/demo/mnist-fashion"
  ],
  "definition": [
    "GetFiles"
  ],
  "registration": {
    "id": "2397f50f-c3cc-49e4-8b61-d5881c35a432",
    "name": "fashion_ds",
    "version": 1,
    "description": "image and label files from fashion mnist",
    "workspace": "Workspace.create(name='basic-ws-2', subscription_id='790907ce-38df-431c-9fce-11b9ec68781d', resource_group='test')"
  }
}

In [17]:


run.find_step_run('train step')[0].register_model(model_name = 'keras-model', model_path = 'outputs/model/', 
                                                  datasets =[('train test data',fashion_ds)])



Model(workspace=Workspace.create(name='basic-ws-2', subscription_id='790907ce-38df-431c-9fce-11b9ec68781d', resource_group='test'), name=keras-model, id=keras-model:1, version=1, tags={}, properties={})